# Using the qiskit interface to perform mGST

In this notebook we will generate random sequences of gates, create qiskit circuits from them, and simulate them on a qiskit simulator.
Then we estimate the gates.

In [12]:
%load_ext autoreload
%autoreload 2

import numpy as np

from qibo import gates

from mGST import qibo_interface
from mGST.qiskit_interface import job_counts_to_mgst_format, get_gate_estimation


backend = Aer.get_backend("qasm_simulator")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
# Example usage with RX, RY, and RZZ gates
rx_pi_2 = gates.RX(0, np.pi / 2)
rx_pi = gates.RX(0, np.pi)
ry_pi_2 = gates.RY(0, np.pi / 2)
ry_pi = gates.RY(0, np.pi)
rzz_pi_2 = gates.RZZ(0, 1, np.pi / 2)
idle_gate = gates.I()

gate_set = [idle_gate, rx_pi, ry_pi, rx_pi_2, ry_pi_2,]
K_true = qibo_interface.qiskit_gate_to_operator(gate_set)

In [14]:
sequence_number = 100  # Number of sequences
shots = 1000  # Number of samples per sequences
sequence_length = 7 # The depth of randomly generated circuits for GST
n_qubits = 1 # The total number of qubits on the backend
active_qubits = [0] # The qubits on which the gate set is defined
n_povm = 2**(len(active_qubits)) # The number of measurement outcomes

In [15]:
# Running the circuits and formating the result
gate_sequences = qibo_interface.get_gate_sequence(sequence_number, sequence_length, len(gate_set))
qiskit_circuits = qibo_interface.get_qiskit_circuits(gate_sequences, gate_set, n_qubits, active_qubits)
job_counts = backend.run(qiskit_circuits, shots=shots).result().get_counts()
sequence_results = job_counts_to_mgst_format(active_qubits, n_povm, job_counts)

In [16]:
# Running GST
X_opt, E_opt, rho_opt = get_gate_estimation(gate_set, gate_sequences, sequence_results, shots, rK = 4)

Starting mGST optimization...
  1%|▊                                                                                 | 1/100 [00:01<02:30,  1.52s/it]
Batch optimization successful, improving estimate over full data....
 12%|█████████▉                                                                         | 6/50 [00:07<00:58,  1.33s/it]
	 Convergence criterion satisfied
	 Final objective 1.24e-4 with # of initializations: 1 in time 9.52s
Mean variation error: 0.026602788732712417
Optimizing gauge...
Compressive GST routine complete
First results:
           F_avg  Min. Spectral distances
Gate 0  0.999692                 0.003428
Gate 1  0.999264                 0.002527
Gate 2  0.999669                 0.001694
Gate 3  0.999072                 0.003611
Gate 4  0.999856                 0.002295
                                
Final cost function     0.000124
Mean TVD estimate-data  0.007763
Mean TVD target-data    0.007470
SPAM error:             0.012498
